In [13]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Настройки подключения
auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster([
    '93.175.29.107',
    '93.175.29.108',
    '93.175.29.109',
    '93.175.29.110',
    '93.175.29.111',
    '93.175.29.114',
    '93.175.29.115',
    '93.175.29.116'
], auth_provider=auth_provider)
session = cluster.connect()

session.execute("""
CREATE KEYSPACE IF NOT EXISTS cassandra_python_demo 
WITH REPLICATION = { 
    'class': 'NetworkTopologyStrategy', 
    'dc1': 3 
}
""")

# Выбор Keyspace
session.set_keyspace('cassandra_python_demo')



In [14]:

# Test Read
print("\n=== Reading Data ===")
rows = session.execute("SELECT * FROM system_schema.keyspaces;")
for row in rows:
    print(row)


=== Reading Data ===
Row(keyspace_name='for2021001', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '2')]))
Row(keyspace_name='mila_hw', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.NetworkTopologyStrategy'), ('dc1', '3')]))
Row(keyspace_name='killrvideo', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '1')]))
Row(keyspace_name='hob2023001_test', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '1')]))
Row(keyspace_name='korzh_space', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.NetworkTopologyStrategy'), ('dc1', '3')]))
Row(keyspace_name='for2021004', durable_writes=True, replication=OrderedMapSerializedKey([('

In [15]:
# Создание Таблицы
session.execute("""
CREATE TABLE IF NOT EXISTS users (
    id UUID PRIMARY KEY,
    name TEXT,
    age INT,
    city TEXT
)
""")

In [16]:
rows = session.execute("""
    SELECT table_name 
    FROM system_schema.tables 
    WHERE keyspace_name = 'python_demo';
""")

print("\n=== Tables in Keyspace 'python_demo' ===")
for row in rows:
    print(row.table_name)


=== Tables in Keyspace 'python_demo' ===
users


In [18]:
from uuid import uuid4

# Вставка данных
user_id = uuid4()
session.execute("""
INSERT INTO users (id, name, age, city) 
VALUES (%s, %s, %s, %s)
""", (user_id, 'Ivan Fedotov', 30, 'Berlin'))



In [23]:
# Чтение данных
print("\n=== Чтение данных ===")
rows = session.execute("SELECT * FROM users")
for row in rows:
    print(row)




=== Чтение данных ===
Row(id=UUID('056d6913-64e4-430b-9026-47300ae1fcd6'), age=29, city='Munich', name='Maria Smith')
Row(id=UUID('0ff332e0-51e8-43c2-95a1-2f500e1eaced'), age=25, city='Berlin', name='Alex Johnson')
Row(id=UUID('23f30e21-af72-4c1a-84a2-470943b462f1'), age=30, city='Munich', name='Ivan Fedotov')
Row(id=UUID('d7b4cc3b-b0dd-4f23-94f1-22ef8ad7d83f'), age=25, city='Berlin', name='Alex Johnson')
Row(id=UUID('4439da6c-f41a-4cf5-a03a-4e41feff1526'), age=29, city='Munich', name='Maria Smith')
Row(id=UUID('f136499c-a2e9-44b9-a3f4-6985106fe7da'), age=30, city='Munich', name='Ivan Fedotov')
Row(id=UUID('01225e68-2e3e-4092-a6c1-8378c2c324fd'), age=30, city='Berlin', name='Ivan Fedotov')


In [20]:
# Обновление данных
session.execute("""
UPDATE users SET city = %s WHERE id = %s
""", ('Munich', user_id))

print("\n=== После обновления ===")
rows = session.execute("SELECT * FROM users")
for row in rows:
    print(row)


=== После обновления ===
Row(id=UUID('0ff332e0-51e8-43c2-95a1-2f500e1eaced'), age=25, city='Berlin', name='Alex Johnson')
Row(id=UUID('23f30e21-af72-4c1a-84a2-470943b462f1'), age=30, city='Munich', name='Ivan Fedotov')
Row(id=UUID('4439da6c-f41a-4cf5-a03a-4e41feff1526'), age=29, city='Munich', name='Maria Smith')
Row(id=UUID('f136499c-a2e9-44b9-a3f4-6985106fe7da'), age=30, city='Munich', name='Ivan Fedotov')
Row(id=UUID('01225e68-2e3e-4092-a6c1-8378c2c324fd'), age=30, city='Berlin', name='Ivan Fedotov')


In [21]:
# чтение порциями
# Установка размера страницы (количество записей на одну порцию)
session.default_fetch_size = 2  

# Подготовленный запрос
statement = session.prepare("SELECT * FROM users")

# Выполнение запроса с постраничной загрузкой
print("\n=== Пагинация ===")
result = session.execute(statement)

# Печать первой страницы
while True:
    for row in result.current_rows:
        print(row)
    if result.has_more_pages:
        input("Press Enter to load more...")
        result.fetch_next_page()
    else:
        break




=== Пагинация ===
Row(id=UUID('0ff332e0-51e8-43c2-95a1-2f500e1eaced'), age=25, city='Berlin', name='Alex Johnson')
Row(id=UUID('23f30e21-af72-4c1a-84a2-470943b462f1'), age=30, city='Munich', name='Ivan Fedotov')
Press Enter to load more...
Row(id=UUID('4439da6c-f41a-4cf5-a03a-4e41feff1526'), age=29, city='Munich', name='Maria Smith')
Row(id=UUID('f136499c-a2e9-44b9-a3f4-6985106fe7da'), age=30, city='Munich', name='Ivan Fedotov')
Press Enter to load more...
Row(id=UUID('01225e68-2e3e-4092-a6c1-8378c2c324fd'), age=30, city='Berlin', name='Ivan Fedotov')


In [22]:
# Создание батча
from cassandra.query import BatchStatement

batch = BatchStatement()
user_id_1 = uuid4()
user_id_2 = uuid4()

batch.add(session.prepare("""
    INSERT INTO users (id, name, age, city) VALUES (?, ?, ?, ?)
"""), (user_id_1, 'Alex Johnson', 25, 'Berlin'))

batch.add(session.prepare("""
    INSERT INTO users (id, name, age, city) VALUES (?, ?, ?, ?)
"""), (user_id_2, 'Maria Smith', 29, 'Munich'))

# Выполнение батч-запроса
session.execute(batch)
print("Batch insert completed.")

Batch insert completed.


In [24]:

# Закрытие соединения
cluster.shutdown()